In [1]:
%run ../../../utils/commonImports.py
%run ../../../utils/tradingImports.py
%matplotlib inline

import sys
sys.path.append(os.path.abspath("../../../modeling/07-base-strategy-03"))

from base import supres, BaseStrategy
from trading.walletHistory import WalletHistory
from exputils import *

# Load Data

In [2]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'
features = ['support']
pair = 'BTC_VTC'

data_train = read_train(pair, dropbox_dir)
ohlc_supports_train_dict = load_model('../../../modeling/06-base-strategy-02/ohlc_supports_train.pkl')

data_test = read_test(pair, dropbox_dir)
ohlc_supports_test_dict = load_model('../../../modeling/06-base-strategy-02/ohlc_supports_test.pkl')

# Prepare last base strategy to get trades

In [3]:
def get_strategy(data, pair, willing_loss, _return, loss_percentage):
    contract_pair = ContractPair.new(*[x.lower() for x in pair.split('_')])
    dm = OhlcDataManager(data[['open', 'low', 'high', 'close']], data[features])
    wallet = {'btc': 100}
    exchange = BackTestExchange(dm, wallet, 0.0025)
    strategy = BaseStrategy(exchange, 
                            dm, 
                            contract_pair, 
                            willing_loss=willing_loss,  
                            target_return=_return,
                            loss_percentage=loss_percentage
                           )
    strategy.trade()
    return strategy

In [4]:
window_size = 4
willing_loss = 0.05
_return = -0.01
loss_percentage = 1.0

In [5]:
data = data_train
data['support'] = ohlc_supports_train_dict[pair][window_size]
strategy_train = get_strategy(data, pair, willing_loss, _return, loss_percentage)
strategy_train.stats('btc').report()
save_model(strategy_train, 'vtc-strategy-train.pkl')

In [6]:
data = data_test
data['support'] = ohlc_supports_test_dict[pair][window_size]
strategy_test = get_strategy(data, pair, willing_loss, _return, loss_percentage)
save_model(strategy_test, 'vtc-strategy-test.pkl')
strategy_test.stats('btc').report()

,btc
startAmount,100.0000
numberOfTrades,293.0000
totalProfit,186.2407
avgProfit,0.6356
winPercentage,79.1809
avgWinTrade,1.3043
avgLossTrade,-1.9075
profitFactor,2.6006
maxDrawdown,8.5457
avgTimeToClose,8.7116
